# Import the required libraries

In [10]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import cv2
import numpy as np
import csv
from PIL import Image
import glob
import os
from skimage import measure
from scipy import ndimage
from tqdm import tqdm

# Config

In [11]:
path = 'dataset_2_2023_05_04'
device = "cuda"
min_overlap_percentage = 85
default_ratio = 3.5
min_pixel = 8000
max_pixel = 25000

# Check the segmentation folder

In [12]:
if not os.path.exists(os.path.join(path,'segmentation_images')):
    os.mkdir(os.path.join(path,'segmentation_images'))

filelist = [ f for f in os.listdir(os.path.join(path,'segmentation_images'))]
for f in filelist:
    os.remove(os.path.join(path,'segmentation_images', f))

# Load the model

In [13]:
sam = sam_model_registry["default"](checkpoint="sam_vit_h_4b8939.pth")
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 15.73 GiB total capacity; 2.79 GiB already allocated; 39.56 MiB free; 2.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Path to the images

In [ ]:
img_path_list = sorted(glob.glob(path+'/train/*')) # normaly glob.glob(path+'/train/*')

# Functions for the loop

In [ ]:
def count_true_values(mask):
    return np.sum(mask)

In [ ]:
def find_biggest_contiguous_area(array):
    labels,_ = ndimage.label(array)
    counts = np.bincount(labels.flatten())
    max_label = np.argmax(counts[1:]) + 1
    region = (labels == max_label)
    
    return region

In [ ]:
def is_mask_inside(large_mask, small_mask):
    indices = np.where(small_mask)
    return np.all(large_mask[indices])

# Loop 

In [ ]:
for img_path in tqdm(img_path_list):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    masks = mask_generator.generate(img)
    sorted_masks = sorted(masks, key=(lambda x: x['area']), reverse=True)
    intersection = []
    for i in range(4): # 4, because the board should always be in the first 4 masks
        maybe_board_mask = sorted_masks[i]['segmentation']
        nb_intersection = 0
        for j in range(len(sorted_masks)-1):
            if is_mask_inside(maybe_board_mask, sorted_masks[j]['segmentation']):
                nb_intersection += 1
        intersection.append(nb_intersection)
    board_mask = sorted_masks[np.argmax(intersection)]['segmentation']

    inside_masks = []
    for i in range(2, len(sorted_masks)):
        small_mask = sorted_masks[i]['segmentation']
        num_small_mask_true = np.sum(small_mask)
        required_overlap = int(num_small_mask_true * min_overlap_percentage / 100)
        indices = np.where(small_mask)
        num_overlapping_true = np.sum(board_mask[indices])

        if num_overlapping_true >= required_overlap:
            inside_masks.append(small_mask)
    
    inside_masks_sorted = sorted(inside_masks, key=count_true_values, reverse=True)

    ratio = []
    for i in range(len(inside_masks_sorted)-1):

        maybe_plug_mask = inside_masks_sorted[i]
        maybe_plug_mask = find_biggest_contiguous_area(maybe_plug_mask)
        pixels = count_true_values(maybe_plug_mask)
        if pixels < min_pixel or pixels > max_pixel:
            ratio.append(0)
            continue
        contours = measure.find_contours(maybe_plug_mask, 0.5)
        width = np.max([np.max(contour[:, 1]) for contour in contours])
        height = np.max([np.max(contour[:, 0]) for contour in contours])
        ratio.append(width/height)

    #closest_value = min(ratio, key=lambda x: abs(x - default_ratio))
    closest_value = max(ratio)
    closest_index = ratio.index(closest_value)
    plug_mask = inside_masks_sorted[closest_index]
    plug_mask = find_biggest_contiguous_area(plug_mask)
    plug_image = Image.fromarray(plug_mask.astype('uint8') * 255, mode='L')
    img_name = img_path.split('/')[-1]
    plug_image.save(path+'/segmentation_images/'+img_name)



  0%|          | 0/2400 [00:00<?, ?it/s]


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

# Sort out the wrong masks

# Test the mask on one image

In [ ]:
number = 100

mask = Image.open(path+'/segmentation_images/picture_'+str(number)+'.png').convert('1')
mask = np.array(mask)

img = cv2.imread(path+'/train/picture_'+str(number)+'.png') # normaly /train/ instead of /test/
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

result_array = np.zeros_like(img)
result_array[mask] = img[mask]
result_image = Image.fromarray(result_array)
display(result_image)
